In [7]:
import pandas as pd
import re

In [8]:
INPUT_CSV = 'data/fashion_2023.csv'
OUTPUT_JSON = 'data/fashion_2023.json'

In [9]:
df = pd.read_csv(INPUT_CSV, encoding="utf-8")
# remove unnamed columns (empty columns)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df

,COMPANY,COUNTRY,1. POLICY & COMMITMENTS,1.1 What are the company's human rights and environmental policies?,Animal Welfare,Annual Leave & Public Holidays,"Anti-bribery, Corruption, & Presentation of False Information",Biodiversity & Conservation,Community Engagement,Discrimination,...,"Publishes annual carbon footprint or GHG emissions in owned and operated facilities (e.g. head office, retail stores, distribution centres, warehouses, transport and mail orders, etc.)","Publishes annual value chain/scope 3 carbon footprint, focusing on detailed calculations on GHG emissions in the supply chain (e.g. at manufacturing and processing facilities, textile production), with estimations for downstream impacts","Publishes annual value chain/carbon footprint at a raw material level, with estimations for downstream impacts","Discloses data on absolute energy reduction in the supply chain (e.g. at manufacturing and processing facilities, fibre production level)","% of energy use coming from renewable sources in the company's owned and operated facilities (e.g. head office, retail stores, distribution centres, warehouses, etc.)","% of energy use coming from renewable sources in the supply chain (e.g. at manufacturing and processing facilities, fibre production level)",Discloses commitment to RE100,"Discloses what proportion of production is powered by coal, including which geographies and sectors are affected",Total Section 5.6,Total Score Section 5
0,Abercrombie & Fitch,USA,NaN,NaN,0.25,0.00,0.25,0.25,0.25,0.25,...,1,0,0,0,0,0,0,0,2,10
1,Adidas,Germany,NaN,NaN,0.25,0.00,0.25,0.25,0.25,0.25,...,1,2,2,0,0,0,0,1,11,35
2,Aeropostale,USA,NaN,NaN,0.00,0.00,0.25,0.00,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
3,AJIO,India,NaN,NaN,0.00,0.00,0.25,0.25,0.25,0.25,...,1,0,0,0,0,0,0,0,1,1
4,ALDI Nord,Germany,NaN,NaN,0.25,0.25,0.00,0.25,0.25,0.25,...,1,0,0,0,1,0,0,0,4,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Wrangler,USA,NaN,NaN,0.25,0.00,0.25,0.25,0.25,0.25,...,0,0,0,0,0,0,0,0,1,8
246,Youngor,China,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
247,Zalando,Germany,NaN,NaN,0.25,0.00,0.25,0.25,0.25,0.25,...,1,2,2,0,1,0,0,0,10,28
248,Zara,Spain,NaN,NaN,0.25,0.00,0.25,0.25,0.25,0.25,...,1,2,2,0,1,0,0,0,14,52


In [10]:
from collections import Counter
country_count = Counter(df['COUNTRY'])
data = []
for _, row in df.iterrows():
    result = {}
    main_category = None
    sub_category = None
    c = 0
    for key, value in row.items():
        key_name = key.strip().lower()
        if key_name.startswith("total"):
            continue
        if key_name == 'company':
            result['Company'] = value
        elif key_name == 'country':
            result['Country'] = value
        elif re.match(r'^\d.[^\d]', key_name):
            main_category = key.split('\n')[0].strip()
            result[main_category] = {}
        elif re.match(r'^\d.\d.', key_name):
            sub_category = key.strip()
            result[main_category][sub_category] = {}
        else:
            result[main_category][sub_category][key.strip()] = value
    data.append(result)

In [11]:
data[0]

{'Company': 'Abercrombie & Fitch',
 'Country': 'USA',
 '1. POLICY & COMMITMENTS': {"1.1 What are the company's human rights and environmental policies?": {'Animal Welfare': 0.25,
   'Annual Leave & Public Holidays': 0.0,
   'Anti-bribery, Corruption, & Presentation of False Information': 0.25,
   'Biodiversity & Conservation': 0.25,
   'Community Engagement': 0.25,
   'Discrimination': 0.25,
   'Diversity & Inclusion': 0.25,
   'Energy & Greenhouse Gas Emissions': 0.25,
   'Equal Pay': 0.0,
   'Freedom of Association, Right to Organise & Collective Bargaining': 0.0,
   'Harassment & Violence': 0.25,
   'Health & Safety': 0.25,
   'Maternity Rights & Parental Leave': 0.0,
   'Mental Health & Wellbeing': 0.0,
   'Restricted Substance List (RSL)': 0.25,
   'Wages & Financial Benefits (e.g. bonuses, insurance, social security, pensions)': 0.25,
   'Waste & Recycling (Packaging/Office/Retail)': 0.25,
   'Waste & Recycling (Product/Textiles)': 0.0,
   'Water Consumption': 0.25,
   'Working H

In [13]:
import json
with open(OUTPUT_JSON, 'w') as f:
    json.dump(data, f, indent=4)